In [ ]:
def difference_in_days(melt, lags, ffday, customer_id_col, create_date_col, net_amount_col):
    for i in range(ffday, lags+1):
        melt['Last-'+str(i)+'day_Sales'] = melt.groupby([customer_id_col])[net_amount_col].shift(i)
    
    melt = melt.reset_index(drop=True)
    
    #for i in range(ffday, lags+1):
        #melt['Last-'+str(i)+'day_Diff']  = melt.groupby([customer_id_col])['Last-'+str(i)+'day_Sales'].diff()
    melt = melt.fillna(0)
    return melt

In [ ]:
import requests
import pandas as pd
from flask import Flask,jsonify, request

def addData(harsh,ordercreationDate):
    servlet_url = 'http://localhost:8080/Backend/PredictData'
    
    # Parameters to be sent to the servlet
    params = {
        'orderCreationDate': ordercreationDate
    }
    
    # Make the GET request to the Java servlet
    try:
        response = requests.get(servlet_url, params=params)

        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx and 5xx)
    except requests.exceptions.RequestException as e:
        return jsonify({'error': str(e)}), 500
    
    json_data = response.json()
    df = pd.DataFrame(json_data)

    harsh = pd.concat([harsh, df], ignore_index=True)

    return harsh

In [ ]:
import pickle 
import numpy as np
import pandas as pd
import seaborn as sns
from currency_converter import CurrencyConverter
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


def predict(df):
    #data fromating

    #df['orderAmount']=df['orderAmount'].str.replace("-",'')
    #df['releasedCreditValue']=df['releasedCreditValue'].str.replace("-",'')
    #df['orderAmount']=df['orderAmount'].str.replace(",",'.')

    df["orderAmount"]=df["orderAmount"].astype(float)
    #df['releasedCreditValue']=df['releasedCreditValue'].str.replace(",",'.')
    df["releasedCreditValue"]=df["releasedCreditValue"].astype(float)

    harsh=addData(df,df['orderCreationDate'][0]) 
    harsh=harsh.fillna("NaN")

    harsh['orderCreationDate'] = pd.to_datetime(harsh['orderCreationDate'], format='%d-%m-%Y')
    harsh['requestedDeliveryDate'] = pd.to_datetime(harsh['requestedDeliveryDate'], format='%d-%m-%Y')
    a=harsh['orderCreationDate']>harsh['requestedDeliveryDate']
    harsh=harsh.loc[~a]
    
    c=CurrencyConverter(fallback_on_missing_rate=True,fallback_on_wrong_date=True)
    harsh['amount_in_usd']=harsh[~(harsh['orderCurrency'].isin(['HU1','AED','BHD','OMR','KWD','SAR','QAR']))].apply( lambda x: c.convert( x['orderAmount'],x['orderCurrency'], 'USD',x['orderCreationDate']), axis = 1)
    
    harsh["CREDIT_STATUS"]=harsh["creditStatus"].astype(float) 
    harsh=harsh.convert_dtypes()
    harsh["uniqueCustNumber"]= str(harsh["uniqueCustNumber"])
    
    # Separate categorical and numerical columns
    cat=[]
    num=[]
    for i in harsh.columns:
        if harsh[i].dtype=="string" :
            cat.append(i)
        elif harsh[i].dtype=="Int64" or harsh[i].dtype=="Float64":
            num.append(i)
    
    #changeing datatype of catagorical values string to catagory
    #harsh[cat]=harsh[cat].astype('category')
    harsh[num] = np.log(harsh[num]) 

    harsh=difference_in_days(harsh,5,1,"uniqueCustNumber","orderCreationDate","amount_in_usd")

    ##Here I am dropping releasedCreditValue as it has 93% null values
    harsh=harsh.drop("releasedCreditValue",axis =1)
    #Removing this column as this has no use as we are using column "amount_in_usd"
    harsh=harsh.drop("orderAmount",axis=1)
    #now dropping column containg Date 
    harsh=harsh.drop(["orderCreationDate","requestedDeliveryDate"],axis =1)
    #now droping uniqueCustNumber column as it is not usabele for pridiction
    harsh=harsh.drop("uniqueCustNumber",axis =1)

    harsh.replace([np.nan, -np.inf], 0, inplace=True)

    harsh=harsh.drop("amount_in_usd",axis=1)

    harsh1=harsh[['CREDIT_STATUS', 'Last-1day_Sales', 'Last-2day_Sales', 'Last-3day_Sales', 'Last-4day_Sales', 'Last-5day_Sales']]
    
    model = pickle.load(open("final1_model.sav", 'rb'))

    display(harsh1)

    first_row = harsh1.iloc[0:1]

    display(first_row)

    prediction = model.predict(first_row)

    result = np.exp(prediction)
    
    return result

In [ ]:
import pandas as pd

# Provided data
data = {
    "slNo": "1",
    "customerOrderID": "1001",
    "salesOrg": "2001",
    "distributionChannel": "Channel1",
    "division": "Division1",
    "releasedCreditValue": "123.45",
    "purchaseOrderType": "Type1",
    "companyCode": "3001",
    "orderCreationDate": "10-1-2022",
    "orderCreationTime": "12:34:56",
    "creditControlArea": "Area1",
    "soldToParty": "4001",
    "orderAmount": "500.67",
    "requestedDeliveryDate": "01-08-2023",
    "orderCurrency": "USD",
    "creditStatus": "2345",
    "customerNumber": "5001",
    "amountInUsd": "678.90",
    "uniqueCustNumber": "12101239393290"
}

# Convert to DataFrame
if isinstance(data, dict):
    data = [data]
df = pd.DataFrame(data)

In [ ]:
ans = predict(df)
print(ans)

In [ ]:
from flask import Flask,jsonify, request
from flask_cors import CORS, cross_origin
import Smartmodel
app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'application/json'

@app.route("/predict", methods=["POST"])
@cross_origin(supports_credentials=True)
def predictOap():
    data = request.get_json()
    print(data)
    if isinstance(data, dict):
        data = [data]
    df = pd.DataFrame(data)
    ans=predict(df)
    #ans=Smartmodel.predict(df)
    result = ans.tolist()
    return jsonify({"results": result})

if __name__ == '__main__':
    app.run(debug=True,use_reloader=False)